# Confusion matrixes for Spacy Model Testing

In [41]:
def confusion_matrix(docs, model_name, model_path, docs_path):
    '''
    docs: list of names of test set docs from training spacy nlp model
    model_name: name of spacy nlp model to test
    model_path: pathname to model
    docs_path: pathname to docs
    '''
    #import spacy and load model
    
    import spacy 
    import os
    os.chdir(model_path)
    nlp_ner = spacy.load(model_name)
    
    #change directory to doc path
    
    os.chdir(docs_path)
    
    #define ann to dict function
    
    def ann_to_dict(nct_id): #took out file path arg
        '''
        nct_id: file name before .ann or .txt
        file_path: pathname to where .ann and .txt files are stored
        return: formatted dictionary
        '''
    
        # change directory
        #os.chdir(file_path)

        # read in files
        with open(nct_id+'.ann') as f1:
            ann=f1.read()
            f1.close()
        with open(nct_id+'.txt') as f2:
            txt=f2.read()
            f2.close()

        # ANN FILE MANIPULATION
        lines=ann.split('\n')
        ents=[]
        for i in range(len(lines)): 
            line=lines[i].split('\t') 
            if 'T' in line[0]:
                try: 
                    start=txt.index(line[2])
                    end= start+len(line[2])
                    label=line[1].split(' ')[0]
                    ents.append((start, end, label))
                except:
                    None

        content={'entities': ents, 'text': txt}

        return content
    # intialize true positive, false positive, false negative variables
    tp=0
    fp=0
    fn=0
    
    # parse through docs and add to confusion matrix variables
    for x in docs: 
        tp_batch=0
        trial=ann_to_dict(x)
        doc = nlp_ner(trial['text'])
        pred_entities = [(e.start_char, e.end_char, e.label_) for e in doc.ents]
        
        for i in trial['entities']:
            if i in pred_entities:
                tp_batch+=1
                
        #update params
        tp+=tp_batch    
        fp+=len(pred_entities)-tp_batch
        fn+=len(trial['entities'])-tp_batch
    
    #print results:
    print('True Positives:')
    print(tp)
    print('False Positives:')
    print(fp)
    print('False Negativies:')
    print(fn)
    
    return [tp, fp, fn]

In [46]:
confusion_matrix(test_docs, 'chia_core', '/Users/meldrumapple/Desktop/Capstone/chia_mod', '/Users/meldrumapple/Desktop/Capstone/chia_with_scope')

True Positives:
3733
False Positives:
190
False Negativies:
2359


[3733, 190, 2359]